In [8]:
import os
import sys

import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors
from hydra import initialize, compose
import pickle
from glob import glob
from pprint import pprint
from tqdm import tqdm
import speclite.filters
from astropy.io import ascii

# set paths to various directories based on the machine this code is being executed on
with initialize(version_base=None, config_path='config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir  = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.lenses.test import SampleStrongLens
from mejiro.plots import diagnostic_plot, plot, plot_util, overplot
from mejiro.analysis import stats
from mejiro.utils import util
from mejiro.helpers import gs, effarea
import mejiro

In [9]:

module_path = os.path.dirname(mejiro.__file__)
filter_responses_dir = os.path.join(module_path, 'data', 'filter_responses')
filter_response_files = sorted(glob(f'{filter_responses_dir}/*.ecsv'))

In [10]:
filter_names = effarea._filter_name_list

In [13]:
filter = filter_names[0]

In [14]:
filter_response_files = sorted(glob(f'{filter_responses_dir}/RomanSCA*-{filter}.ecsv'))

data = ascii.read(filter_response_files[0])

In [27]:
total_data = [ascii.read(f) for f in filter_response_files]

In [18]:
wavelengths = []
responses = []

for file in filter_response_files:
    data = ascii.read(file)
    wavelengths.append(data['wavelength'])
    responses.append(data['response'])

In [25]:
min_wave = np.min([np.min(w.value) for w in wavelengths])
max_wave = np.max([np.max(w.value) for w in wavelengths])

print(f'Min wave: {min_wave}, Max wave: {max_wave}')

Min wave: 9880.0, Max wave: 19470.0


In [ ]:
total_wavelengths = np.arange(min_wave, max_wave, 10)
total_responses = []

In [20]:
for wavelength in wavelengths:
    print(len(wavelength))

953
952
949
952
949
949
948
949
947
953
952
949
951
951
948
949
949
950


In [15]:
data

wavelength,response
Angstrom,
float64,float64
4730.0,0.0
4740.0,2.2104853207207685e-05
4750.0,0.00011052426603603843
4760.0,0.0006189358898018152
4770.0,0.002586267825243299
4780.0,0.008598787897603788
4790.0,0.023254305573982486
4800.0,0.05143799341317228


In [16]:
for filter in filter_names:
    filter_response_files = sorted(glob(f'{filter_responses_dir}/RomanSCA*-{filter}.ecsv'))